Importing libraries

In [1]:
import requests
import pandas as pd
import numpy as np
import urllib
import operator

Downloading training and testing files from the internet

In [2]:
url1 = "http://kevincrook.com/utd/market_basket_training.txt"
r1 = requests.get(url1)
with open('market_basket_training.txt', 'wb') as output:
    output.write(r1.content)
train_data = pd.read_csv('market_basket_training.txt',names = ["transaction","item1","item2","item3","item4"], header=None)   
url2 = "http://kevincrook.com/utd/market_basket_test.txt"
r2 = requests.get(url2)
with open('market_basket_test.txt', 'wb') as output:
    output.write(r2.content)

Making dataframes for training and testing files

In [3]:


test_data = pd.read_csv('market_basket_test.txt',names = ["transaction","item1","item2","item3","item4"], header=None)

Manipulating training dataframe to combine items and make set out of it

In [4]:
train_data_concat = train_data.fillna('')
train_data_concat['all_items'] = train_data_concat['item1'] + ';' + train_data_concat['item2'] + ';' + train_data_concat['item3'] + ';' + train_data_concat['item4']
train_data_concat['all_items_set'] = train_data_concat.all_items.str.strip(';').str.split('\s*;\s*').apply(set)

getting the count of each item combination

In [5]:
df_count = train_data_concat.groupby('all_items').size()
df_count = df_count.to_frame().reset_index()
df_count.columns = ['all_items','count']

combining both the dataframes to create a final dataframe with sets and count

In [6]:
df_train_data = train_data_concat.drop_duplicates('all_items')
df_final = pd.merge(df_train_data, df_count, on='all_items', how='outer')

Manipulating test dataframe to combine items and make set out of it

In [7]:
test_data_concat = test_data.fillna('')
test_data_concat['all_items'] = test_data_concat['item1'] + ';' + test_data_concat['item2'] + ';' + test_data_concat['item3'] + ';' + test_data_concat['item4']
test_data_concat['all_items_set'] = test_data_concat.all_items.str.strip(';').str.split('\s*;\s*').apply(set)

logic for getting the recommendations

In [8]:
item_freq1 = test_data_concat.copy()
item_freq1['rec_item'] = ''

for i in range(len(test_data_concat)):
    freq = dict()
    for j in range(len(df_final)):
        if test_data_concat.iloc[i,6].issubset(df_final.iloc[j,6]):
            if len(test_data_concat.iloc[i,6]) == len(df_final.iloc[j,6]) - 1:
                freq[tuple(df_final.iloc[j,6])] = df_final.iloc[j,7]
    s_freq = sorted(freq.items(), key=operator.itemgetter(1),reverse=True)
    if s_freq != []:
        r = set(s_freq[0][0]) - test_data_concat.iloc[i,6] 
        item_freq1['rec_item'][i] = r
    else:
        for elem in test_data_concat.iloc[i,6]:
            for j in range(len(df_final)):
                if (test_data_concat.iloc[i,6] - {elem}).issubset(df_final.iloc[j,6]):
                    if len(df_final.iloc[j,6]) == len((test_data_concat.iloc[i,6] - {elem})) + 1:
                        freq[tuple(df_final.iloc[j,6])] = df_final.iloc[j,7]
        s_freq = sorted(freq.items(), key=operator.itemgetter(1),reverse=True)
        if s_freq != []:
            r = set(s_freq[0][0]) - test_data_concat.iloc[i,6]
            item_freq1['rec_item'][i] = r
        else:
            for elem in test_data_concat.iloc[i,6]:
                for j in range(len(df_final)):
                    if ({elem}).issubset(df_final.iloc[j,6]):
                        if len(df_final.iloc[j,6]) == 2:
                            freq[tuple(df_final.iloc[j,6])] = df_final.iloc[j,7]
            s_freq = sorted(freq.items(), key=operator.itemgetter(1),reverse=True)
            if s_freq != []:
                r = set(s_freq[0][0]) - test_data_concat.iloc[i,6]
                item_freq1['rec_item'][i] = r

C:\Users\akash\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\akash\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akash\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


writing outputs and creating recommendation text file using the 

In [9]:
df_out = item_freq1[['transaction','rec_item']]
df_out['rec_item'] = df_out['rec_item'].apply(lambda x: ''.join(str(e) for e in x))
df_out['transaction'] = df_out['transaction'].apply(lambda x: '{0:0>3}'.format(x))
df_out.to_csv('market_basket_recommendations.txt', sep=',', header=None, index=False, encoding='utf-8')

C:\Users\akash\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\akash\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
